# liquidity model preference

## Organize CSVs into time-series dataframe

In [1]:
# Imports
import os
import re
import logging
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer, InputExample, losses, util
from torch.utils.data import DataLoader
import nltk
import torch

/Users/alexchen/Downloads/Projects/new_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Setup
nltk.download('punkt')
nltk.download('stopwords')
logging.basicConfig(level=logging.WARNING)

[nltk_data] Downloading package punkt to /Users/alexchen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexchen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Paths
batch1_labeled_path = "/Users/alexchen/Downloads/Projects/vc-research/URAP VC Research - [Readable] Batch 1 Main.csv"
batch1_lp_path = "/Users/alexchen/Downloads/Projects/vc-research/URAP VC Research - Batch 1 Details.csv"
txt_folder_path = "/Users/alexchen/Downloads/Projects/vc-research/Batch1_text_readable"

In [4]:
# Load data
batch1_labeled = pd.read_csv(batch1_labeled_path)
batch1_lp = pd.read_csv(batch1_lp_path)

## Part 1. Classify which text files contain liquidation preference or not

In [5]:
# --- Preprocessing ---
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def prepare_multiindex(df, date_col="Date", group_cols=["Company Name"]):
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.set_index(group_cols + [date_col]).sort_index()
    df = df.groupby(level=0, sort=False).apply(lambda x: x.sort_index(level=1))
    df.index = df.index.droplevel(0)
    return df

batch1_labeled_multiindex = prepare_multiindex(batch1_labeled)
batch1_lp_multiindex = prepare_multiindex(batch1_lp)

In [6]:
# --- Load and preprocess text data ---
text_data, labels, document_names = [], [], []
for _, row in batch1_labeled.iterrows():
    file_name = row['File Name']
    label = row['Contains Liquidity Preference']
    file_path = os.path.join(txt_folder_path, file_name + ".txt")
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as file:
            text = clean_text(file.read())
            text_data.append(text)
            labels.append(label)
            document_names.append(file_name)
    else:
        logging.warning(f"File not found: {file_path}")

In [7]:
# --- Split and vectorize text ---
X_train, X_test, y_train, y_test, train_docs, test_docs = train_test_split(
    text_data, labels, document_names, test_size=0.25, stratify=labels, random_state=42
)

vectorizer = TfidfVectorizer(ngram_range=(1,3), max_features=2000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

bert_model = SentenceTransformer('all-MiniLM-L6-v2')
X_train_bert = [bert_model.encode(sent_tokenize(doc)).mean(axis=0) for doc in X_train]
X_test_bert = [bert_model.encode(sent_tokenize(doc)).mean(axis=0) for doc in X_test]

X_train_combined = np.hstack([X_train_tfidf.toarray(), np.array(X_train_bert)])
X_test_combined = np.hstack([X_test_tfidf.toarray(), np.array(X_test_bert)])

In [8]:
# --- Train RandomForest ---
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

rf_model = GridSearchCV(RandomForestClassifier(random_state=42, class_weight="balanced"), param_grid, cv=5, n_jobs=-1)
rf_model.fit(X_train_combined, y_train)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200]})

In [9]:
# --- Evaluate Model ---
y_pred = rf_model.best_estimator_.predict(X_test_combined)
y_pred_prob = rf_model.best_estimator_.predict_proba(X_test_combined)

predictions_df = pd.DataFrame({
    'Document': test_docs,
    'True Classification': y_test,
    'Predicted Classification': y_pred,
    'Confidence Score': y_pred_prob[:, 1]
})

In [10]:
predictions_df

,Document,True Classification,Predicted Classification,Confidence Score
0,48_2013-12-06_Certificates of Incorporation,1,1,0.995
1,27_2004-08-17_Certificates of Incorporation,0,0,0.000
2,27_2006-08-30_Certificates of Incorporation,1,1,0.985
3,24_2004-12-01_Certificates of Incorporation,0,0,0.075
4,16_2015-04-22_Certificates of Incorporation,0,0,0.015
5,16_2007-05-16_Certificates of Incorporation,1,1,1.000
6,81_2010-03-17_Certificates of Incorporation,0,0,0.015
7,81_2010-06-10_Certificates of Incorporation,1,1,0.995
8,35_2018-02-23_Certificates of Incorporation,1,1,1.000
9,16_2009-01-20_Certificates of Incorporation,1,1,1.000


## Label and extract relevant sentences

In [11]:
# Load base BERT model
bert_model = SentenceTransformer('all-MiniLM-L6-v2')

# --- Define Property Tags and Expanded Heuristics ---
PROPERTY_TAGS = ['Company Name', 'Date', 'Document Type', 'Preferred Stocks', 'Priority Order', 'Liquidation Value']
KEYWORDS = {
    'Company Name': ["certificate of incorporation", "incorporated", "corporation", "company name"],
    'Date': ["filed", "effective date", r"\d{2}/\d{2}/\d{4}"],
    'Document Type': ["certificate of amendment", "articles of incorporation", "amended and restated"],
    'Preferred Stocks': ["preferred stock", "series a", "series b"],
    'Priority Order': [
        "prior and in preference", "ranking junior", "paid before", "in preference to", "senior to",
        "shall be entitled to be paid", "after payment has been made", "in liquidation, before any distribution",
        "subject to", "subordinated to", "senior liquidation rights", "distribution of assets"
    ],
    'Liquidation Value': ["liquidation preference", "liquidation value", "entitled to", "distribution", r"\$[0-9]+\.?[0-9]*"]
}


In [12]:
# --- Chunk text into overlapping blocks of sentences ---
def chunk_text(text, size=3, stride=1):
    sentences = sent_tokenize(text)
    chunks = []
    for i in range(0, len(sentences) - size + 1, stride):
        chunk = " ".join(sentences[i:i + size])
        chunks.append(chunk)
    return chunks

In [13]:
# --- Heuristic Labeling Function (applies to chunks) ---
def label_chunks_heuristically(folder_path):
    data = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8', errors='ignore') as f:
                text = f.read().replace("\n", " ")
                chunks = chunk_text(text, size=3, stride=1)
                for chunk in chunks:
                    tags = []
                    for tag, keywords in KEYWORDS.items():
                        if any(re.search(kw, chunk, re.IGNORECASE) for kw in keywords):
                            tags.append(tag)
                    if tags:
                        data.append({"Filename": filename, "Chunk": chunk, "Labels": ", ".join(tags)})
    return pd.DataFrame(data)

In [14]:
# --- Build Training Examples ---
def build_training_examples(labeled_df):
    examples = []
    for _, row in labeled_df.iterrows():
        for tag in PROPERTY_TAGS:
            chunk = str(row['Chunk'])
            label = 1 if tag in row['Labels'] else 0
            examples.append(InputExample(texts=[chunk, tag], label=float(label)))
    return examples

In [15]:
# --- Fine-Tune the Model ---
def fine_tune_bert_model(bert_model, training_examples):
    train_dataloader = DataLoader(training_examples, shuffle=True, batch_size=32)
    train_loss = losses.CosineSimilarityLoss(model=bert_model)
    bert_model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=1,
        warmup_steps=5,
        show_progress_bar=True
    )
    return bert_model

In [16]:
# --- Classify Chunks by Tag ---
def classify_chunks(chunks, model, threshold=0.5):
    results = defaultdict(list)
    tag_thresholds = {
        'Priority Order': 0.45
    }
    for chunk in chunks:
        for tag in PROPERTY_TAGS:
            score = util.cos_sim(model.encode(chunk), model.encode(tag))[0][0].item()
            if score >= tag_thresholds.get(tag, threshold):
                results[tag].append((chunk, score))
    return results

In [17]:
# --- Extract Tagged Chunks from a Document ---
def extract_relevant_chunks_from_document(text, model, threshold=0.5):
    chunks = chunk_text(text.replace('\n', ' '), size=3, stride=1)
    return classify_chunks(chunks, model, threshold)

In [18]:
# --- Process a Directory of Text Files ---
def process_directory_with_model(folder_path, model, threshold=0.5):
    records = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8', errors='ignore') as f:
                text = f.read()
                results = extract_relevant_chunks_from_document(text, model, threshold)
                record = {"Filename": filename}
                for tag in PROPERTY_TAGS:
                    chunks = results.get(tag, [])
                    record[tag] = " ||| ".join([chunk for chunk, _ in chunks])
                records.append(record)
    return pd.DataFrame(records)

In [19]:
# --- Run the Pipeline ---
folder = "/Users/alexchen/Downloads/Projects/vc-research/Batch1_text_readable"
labeled_data = label_chunks_heuristically(folder)
examples = build_training_examples(labeled_data)
fine_tuned_model = fine_tune_bert_model(bert_model, examples)
extracted_df = process_directory_with_model(folder, fine_tuned_model)
extracted_df

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.098000
1000,0.068500
1500,0.054600


,Filename,Company Name,Date,Document Type,Preferred Stocks,Priority Order,Liquidation Value
0,45_2008-01-17_Certificates of Incorporation.txt,Inc. and the date of filing of the original Ce...,State of Delaware Secre of State Division of C...,State of Delaware Secre of State Division of C...,IV. A. The Company is authorized to issue two ...,Except as otherwise provided herein or as requ...,A. The Company is authorized to issue two clas...
1,16_2015-04-22_Certificates of Incorporation.txt,,State of Delaware Secreta pe arate = Division ...,,,,
2,28_2009-12-17_Certificates of Incorporation.txt,State of Delaware Secre of State Division of C...,State of Delaware Secre of State Division of C...,State of Delaware Secre of State Division of C...,ARTICLE Ill The address of the Corporation’s ...,be designated “Series B Preferred Stock” and 2...,ARTICLE Ill The address of the Corporation’s ...
3,34_2010-01-28_Certificates of Incorporation.txt,of State Division of Corporations Delivered 02...,State of Delaware Secreta. of State Division o...,The amendment to the Certificate of Incorporat...,"Article TV, Paragraph A. of the Certificate of...",,This corporation is authorized to issue two cl...
4,27_2006-08-23_Certificates of Incorporation.txt,A0b493b7 10cs | 1Ho78 cake A | 1 Office of the...,,A0b493b7 10cs | 1Ho78 cake A | 1 Office of the...,Article Oil of the Articles of Incorporation o...,,
...,...,...,...,...,...,...,...
85,28_2007-06-15_Certificates of Incorporation.txt,State of Delaware Secre of State Division of C...,State of Delaware Secre of State Division of C...,State of Delaware Secre of State Division of C...,The name of the registered agent at such addre...,"par value per share, 9.819.635 of which shall ...","City of Wilmington, County of New Castle, 1980..."
86,49_2007-01-23_Certificates of Incorporation.txt,"The name of the corporation is A123 Systems, I...",State of Delaware Secretary of State Division ...,State of Delaware Secretary of State Division ...,THIRD. The nature of the business or purposes ...,"1. General. The voting, dividend and liquidati...",THIRD. The nature of the business or purposes ...
87,16_2006-03-09_Certificates of Incorporation.txt,State of Delaware Secretary of State Division ...,State of Delaware Secretary of State Division ...,State of Delaware Secretary of State Division ...,ARTICLE Ill The nature of the business or purp...,"Notwithstanding the prior sentence, the sale o...",This corporation is authorized to issue two cl...
88,16_2003-07-03_Certificates of Incorporation.txt,The name of its registered agent at such addre...,State of Delaware Secretary of State Division ...,,,,


In [20]:
extracted_df['Preferred Stocks'][40]

"B. By unanimous written consent of the Board of Directors of the Corporation, a resolution was duly adopted, pursuant to Sections 141(f) and 242 of the Delaware General Corporation Law, setting forth an amendment to the Certificate of Incorporation of the Corporation and declaring the amendment to be advisable. C. The amendment as adopted increases the number of shares of Commun Stock  Preferred Stock and Series 0 Preferred Stock the Corporation is authorized to issue. ||| By unanimous written consent of the Board of Directors of the Corporation, a resolution was duly adopted, pursuant to Sections 141(f) and 242 of the Delaware General Corporation Law, setting forth an amendment to the Certificate of Incorporation of the Corporation and declaring the amendment to be advisable. C. The amendment as adopted increases the number of shares of Commun Stock  Preferred Stock and Series 0 Preferred Stock the Corporation is authorized to issue. as set forth below, D. The amendment revises Artic

In [21]:
extracted_df.to_csv('Extracted Sentences - Batch 1.csv', index=False)

ChatGPT Prompt:

Based on the extracted sentences in each of the cells, isolate just the desired information as describe below: 
File Name: Do not modify values in this column
Company Name: Identify and extract the company's name as a string type (Example: 3VR Security INC, The 41st Parameter INC, etc.)
Date: Identify and extract the date when the article was filed as a datetime type (Example: 2004-04-22, 2012-12-17, etc.)
Document Type: Identify and extract the type of document that was submitted as a string type (Example: "Certificate of Incorporation", "Amended and Restated Certificate of Incorporation", etc.) 
Preferred Stock: Identify and extract the unique types of preferred shares as a list of strings (Example: ['Series A', 'Series B', 'Series C', 'Series D'])
Liquidation Value: Identify and extract the dollar liquidation amount for each preferred stock as a list of floats; the length of the list should be the same length as the list for preferred stocks; if the liquidation preference is the original issue price use that value (Example: [0.431469, 0.624136, 0.474550, 0.152430])

Return the result after this extraction in the form of a dataframe and then export as a CSV

## Extract desired information from labeled sentences

In [21]:
# # Load base BERT model
# bert_model = SentenceTransformer('all-MiniLM-L6-v2')

# # --- Define Property Tags ---
# PROPERTY_TAGS = ['Company Name', 'Date', 'Document Type', 'Preferred Stocks', 'Priority Order', 'Liquidation Value']

# # --- Build Training Examples ---
# def build_training_examples(labeled_df):
#     examples = []
#     for _, row in labeled_df.iterrows():
#         for tag in PROPERTY_TAGS:
#             sentence = str(row['Sentence'])
#             label = 1 if tag in row['Labels'] else 0
#             examples.append(InputExample(texts=[sentence, tag], label=float(label)))
#     return examples

# # --- Fine-Tune the Model ---
# def fine_tune_bert_model(bert_model, training_examples):
#     train_dataloader = DataLoader(training_examples, shuffle=True, batch_size=16)
#     train_loss = losses.CosineSimilarityLoss(model=bert_model)
#     bert_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2, warmup_steps=10)
#     return bert_model

# # --- Classify Sentences by Tag ---
# def classify_sentences(sentences, model, threshold=0.5):
#     results = defaultdict(list)
#     for sentence in sentences:
#         for tag in PROPERTY_TAGS:
#             score = util.cos_sim(model.encode(sentence), model.encode(tag))[0][0].item()
#             if score >= threshold:
#                 results[tag].append((sentence, score))
#     return results

# # --- Extract Tagged Sentences from a Document ---
# def extract_relevant_sentences_from_document(text, model, threshold=0.5):
#     sentences = sent_tokenize(text.replace('\n', ' '))
#     return classify_sentences(sentences, model, threshold)

# # --- Process a Directory of Text Files ---
# def process_directory_with_model(folder_path, model, threshold=0.5):
#     records = []
#     for filename in os.listdir(folder_path):
#         if filename.endswith(".txt"):
#             with open(os.path.join(folder_path, filename), 'r', encoding='utf-8', errors='ignore') as f:
#                 text = f.read()
#                 results = extract_relevant_sentences_from_document(text, model, threshold)
#                 record = {"Filename": filename}
#                 for tag in PROPERTY_TAGS:
#                     sentences = results.get(tag, [])
#                     record[tag] = "; ".join([s for s, _ in sentences])
#                 records.append(record)
#     return pd.DataFrame(records)

# # --- Fine-Tune and Apply the Model ---
# labeled_data = pd.read_csv("/Users/alexchen/Downloads/Projects/vc-research/labeled_sentences.csv")
# examples = build_training_examples(labeled_data)
# fine_tuned_model = fine_tune_bert_model(bert_model, examples)
# folder = "/Users/alexchen/Downloads/Projects/vc-research/Batch1_text_readable"
# extracted_df = process_directory_with_model(folder, fine_tuned_model)
# print(extracted_df)

focus on extracting just relevant text right now, not structured in a dataframe
move on to other provisions and getting the relevant text areas
